In [1]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json
from openai import OpenAI

In [2]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        client = OpenAI(base_url="https://api.fanar.qa/v1",api_key=os.environ["FANAR_API_KEY"],)
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="azure"
deployment = APIKeyManager(model_type, "./azure.env")


In [9]:
class PodcastOutlineGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.deployment = deployment
        self.model = model

    def generate_outline(self, topic, information, host_persona, guest_persona):
        prompt = f"""
You are a skilled podcast content planner.

Create an outline for a 10-minute Modern Standard Arabic (MSA) podcast episode.

Context:
- Topic: "{topic}"
- Host persona: "{host_persona}"
- Guest persona: "{guest_persona}"
- Reference information:
  {information}

Output requirements:
- Return **only** valid JSON (no markdown, no comments).
- Keep all keys in English exactly as shown.
- Fill every value in clear Arabic.
- Ensure target_duration_sec = 600 and per-section durations match the schema.


{{
  "meta": {{
    "topic": "عنوان الحلقة التجريبي",
    "target_duration_sec": 600
  }},
  "intro": {{
    "host_open": "أهلاً بكم في هذه الحلقة الجديدة من بودكاستنا.",
    "guest_intro": "يسرّنا استضافة الخبير فلان للحديث حول الموضوع.",
    "teaser": "ابقوا معنا لاكتشاف أبرز النقاط المثيرة.",
    "duration_sec": 60
  }},
  "segments": [
    {{
      "id": 1,
      "title": "أول نقطة نقاش",
      "lead_speaker": "guest",
      "discussion": "ملخص سريع عن النقطة الأساسية.",
      "host_questions": [
        "ما أبرز مثال لديك؟",
        "كيف تؤثر هذه النقطة على المستمعين؟"
      ],
      "response_hint": "أضف قصة واقعية قصيرة.",
      "emotion_tone": "متفائل",
      "duration_sec": 120
    }},
    {{
      "id": 2,
      "title": "ثاني نقطة نقاش",
      "lead_speaker": "host",
      "discussion": "عرض للتحديات المحتملة والحلول.",
      "host_questions": [
        "ما أكبر تحدٍ تواجهه؟",
        "كيف يمكن تجاوز هذا التحدي؟"
      ],
      "response_hint": "أبرز أهمية التعاون.",
      "emotion_tone": "جدي",
      "duration_sec": 120
    }}
  ],
  "outro": {{
    "summary": "استعرضنا أهم الأفكار والتجارب.",
    "closing_question": "ما رأيكم أنتم؟ شاركونا آراءكم.",
    "credits": "شكراً للضيف وشكراً لكم على الاستماع.",
    "duration_sec": 60
  }}
}}

أعد كتابة جميع القيم لتناسب الموضوع المحدد مع الحفاظ على البنية ذاتها.
"""
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a skilled podcast content planner."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content


In [10]:
outline_generator = PodcastOutlineGenerator (deployment)


In [11]:
topic = "الزراعة الحضرية المستدامة في العالم العربي"

information = (
    "تظهر الزراعة الحضرية كحل استراتيجي لتحديات الأمن الغذائي في مدن عربية عديدة. "
    "فالمنطقة العربية تستورد أكثر من 85٪ من احتياجاتها الغذائية، بينما يؤدي التوسع العمراني والمناخ الجاف "
    "إلى ضغوط على الزراعة التقليدية. من المبادرات البارزة مجمّع «بستانيكا» في دبي الذي ينتج مليون كيلوجرام "
    "من الخضروات الورقية سنويًّا مع توفير 95٪ من المياه، وحدائق الأسطح المائية في برنامج «السطح الأخضر» "
    "بالقاهرة، إضافةً إلى مزارع الحاويات المدعومة بصندوق أبوظبي للتكنولوجيا الزراعية بقيمة 100 مليون دولار. "
    "تشمل عوامل الاستدامة الرئيسية نظم الزراعة المائية والهوائية الموفّرة للمياه، والتحكّم المناخي بالطاقة الشمسية، "
    "وممارسات الاقتصاد الدائري لإعادة تدوير المياه الرمادية والنفايات العضوية. "
    "ومع ذلك، تظل التحديات مثل ارتفاع تكاليف الاستثمار، ونقص الخبرة المحلية، والإجراءات التنظيمية المعقّدة "
    "لشهادات سلامة الأغذية، وضعف الوعي المجتمعي. غالبًا ما تنجح المشاريع عندما تتعاون مع الجامعات للبحث والتطوير، "
    "وتستفيد من الحوافز الحكومية مثل التعرفة المخفَّضة للطاقة أو التمويل الأخضر. "
    "ينبغي للنقاش أن يوازن بين الجدوى الاقتصادية والأثر البيئي ونماذج إشراك المجتمع."
)

host_persona  = "مريم أبو زهرة، طالبة صحافة شغوفة بالقصص الإنسانية"
guest_persona = "رانية السيد، عالمة بيئة متخصصة في حماية الشعاب المرجانية"

outline_text = outline_generator.generate_outline(
    topic,
    information,
    host_persona,
    guest_persona
)

print("Generated Outline:\n", outline_text)

Generated Outline:
 {
  "meta": {
    "topic": "الزراعة الحضرية المستدامة في العالم العربي",
    "target_duration_sec": 600
  },
  "intro": {
    "host_open": "أهلاً بكم في هذه الحلقة الجديدة من بودكاستنا، حيث نسلط الضوء على موضوع يهمّ حياتنا اليومية ومستقبلنا المشترك: الزراعة الحضرية المستدامة.",
    "guest_intro": "يسرّني استضافة العالمة البيئية رانية السيد، المتخصصة في حماية الشعاب المرجانية وأحد الأصوات البارزة في دعم المشاريع المستدامة.",
    "teaser": "تابعونا لاستكشاف كيف تغيّر الزراعة الحضرية مستقبل الأمن الغذائي في مدن العالم العربي.",
    "duration_sec": 60
  },
  "segments": [
    {
      "id": 1,
      "title": "ما هي الزراعة الحضرية وأهميتها في العالم العربي؟",
      "lead_speaker": "guest",
      "discussion": "الحديث عن المفهوم العام للزراعة الحضرية وكيف يمكنها أن تصبح حلاً استراتيجياً للتحديات الزراعية والغذائية في المنطقة العربية.",
      "host_questions": [
        "ما هي أبرز الأساليب المستخدمة في الزراعة الحضرية في العالم العربي؟",
        "هل لديك مثال عن مشروع ناج

In [12]:

# class DialogueEnhancer:
#     def __init__(self, deployment,model="gpt-4o"):
#         self.model = model

#     def enhance_section(self, section_name, content, criteria, style_example=None):
#         if style_example:
#             prompt = (
#                 f"""Here is an example of the dialogue style you should follow:\n{style_example}\n\n"
#                 Given the following podcast section, modify it based on the following criteria: {criteria}

#                 Section: {section_name}
#                 Content: {content}

#                 Provide a compelling and engaging script."""
#             )
#         else:
#             prompt = (
#                 f"""Given the following podcast section, modify it based on the following criteria: {criteria}

#                 Section: {section_name}
#                 Content: {content}

#                 Provide a compelling and engaging script."""
#             )
#         response = deployment.chat.completions.create(
#             model=self.model,
#             messages=[
#                 {"role": "system", "content": "You are a professional podcast scriptwriter."},
#                 {"role": "user", "content": prompt}
#             ]
#         )
#         return response.choices[0].message.content
    


# class SpontaneousDialogueGenerator:
#     def __init__(self, deployment, model="gpt-4o"):
#         self.model = model
#     def generate_dialogue(self, section, section_content, host_persona, guest_persona, style_example=None):
#         if style_example:
#             prompt = (
#                 f"""Here is an example of the dialogue style you should follow:\n{style_example}\n\n"
#                 f"Now, transform the following podcast section into a spontaneous dialogue between host {host_persona} and guest {guest_persona}, naturally including conversational fillers, pauses with possible duration in brackets [pause: 2s], and interaction tags like <laughter>, <overlap>, <interruption>, <hesitations>.\n\n"
#                 f"Section: {section}\nContent: {section_content}"""
#             )
#         else:
#             prompt = (
#                 f"""Transform the following structured podcast section into a spontaneous dialogue between a host named {host_persona} and a guest named {guest_persona}, naturally including conversational fillers, pauses with possible duration in brackets [pause: 2s], and interaction tags.

#                 Section: {section}
#                 Content: {section_content}

#                 Include conversational tags like <laughter>, <overlap>, <interruption>, <hesitations> naturally."""
#             )
#         response = deployment.chat.completions.create(
#             model=self.model,
#             messages=[
#                 {"role": "system", "content": "You are an expert at writing spontaneous and realistic podcast dialogues."},
#                 {"role": "user", "content": prompt}
#             ]
#         )
#         return response.choices[0].message.content


#     def generate_dialogue_zeroshot(self, section, section_content, host_persona, guest_persona):
#         prompt = (
#             f"""Transform the following structured podcast section into a spontaneous dialogue between a host named {host_persona} and a guest named {guest_persona}, naturally including conversational fillers, pauses with possible duration in brackets [pause: 2s], and interaction tags.
            
#             Section: {section}
#             Content: {section_content}
            
#             Include conversational tags like <laughter>, <overlap>, <interruption>, <hesitations> naturally and also include speaker identification like Host:<utt>, Guest: <utt>.
#             """
#         )
#         response = deployment.chat.completions.create(
#             model=self.model,
#             messages=[
#                 {"role": "system", "content": "You are an expert at writing spontaneous and realistic podcast dialogues."},
#                 {"role": "user", "content": prompt}
#             ]
#         )
#         return response.choices[0].message.content
    
# dialogue_generator=SpontaneousDialogueGenerator(deployment,"fanar")
# enhancer=DialogueEnhancer(deployment,"fanar")

# outline = json.loads(outline_text)

# print(outline)



# style_examples_dialogues=dialogue_styles['inspirational']
# podcast_dialogues = []
# # Introductions
# for key in ["Intro1", "Intro2"]:
#     enhanced_intro = enhancer.enhance_section(
#         key,
#         " ".join(outline[key]["script"]),
#         criteria="Engaging, energetic, welcoming, and enthusiastic"
#     )
#     podcast_dialogues.append({"section": key, "dialogue": enhanced_intro})

# # Main Discussion
# tp_key = "Points"
# talking_points = outline[tp_key]["talking_points"]
# for tp_title, tp_content in talking_points.items():
#     section_content = f"{tp_title}: {tp_content['discussion']} " + \
#                       " ".join([f"Q: {q}" for q in tp_content["questions"]])
#     if style_examples_dialogues:  
#         spontaneous_dialogue = dialogue_generator.generate_dialogue(
#             tp_title, section_content, host_persona, guest_persona, style_examples_dialogues
#         )
#     else:                
#         spontaneous_dialogue = dialogue_generator.generate_dialogue(
#             tp_title, section_content, host_persona, guest_persona
#         )
#     podcast_dialogues.append({"section": tp_title, "dialogue": spontaneous_dialogue})
# # Enhance conclusion
# conclusion_key = "Con"
# enhanced_conclusion = enhancer.enhance_section(
#     conclusion_key,
#     " ".join(outline[conclusion_key]["script"]),
#     criteria="Powerful, thoughtful, reflective, and thought-provoking"
# )
# podcast_dialogues.append({"section": conclusion_key, "dialogue": enhanced_conclusion})



In [13]:
class DialogueGenerator:
    """
    Generate spontaneous, lifelike Arabic podcast dialogue from an outline section.
    Merges the old DialogueEnhancer + SpontaneousDialogueGenerator logic.
    """

    def __init__(self, deployment, model: str = "gpt-4o"):
        self.deployment = deployment
        self.model = model

    # ------------------------------------------------------------------
    # core method
    # ------------------------------------------------------------------
    def generate_dialogue(
        self,
        section_title: str,
        section_content: str,
        host_persona: str,
        guest_persona: str,
        tone: str = "محايد",
        style_example: str | None = None,
    ) -> str:
        """
        Return an Arabic dialogue (string) with fillers, pauses, and emotion tags.

        Tags supported
        --------------
        [pause:1.5s]     – explicit pause in seconds
        <ضحك> / <تنهد>  – emotion or vocal tag
        Fillers          – أمم، يعني، آآه، بصراحة …
        """
        base_prompt = f"""
Write a spontaneous Modern Standard Arabic podcast dialogue.

Requirements:
• Speakers: Host ({host_persona}) and Guest ({guest_persona}).
• Tone: {tone}.
• Add natural fillers (أمم، يعني، آآه، بصراحة).
• Insert pauses like [pause:1.5s].
• Use emotion tags such as <ضحك> (laugh), <تنهد> (sigh), <دهشة> (gasp).
• Prefix each utterance with "Host:" or "Guest:".
• Aim for 8-12 exchanges unless the section is an intro or outro (then 4–6 exchanges).
• Do NOT include any explanations, markdown, or extra commentary—dialogue only.

Section context:
عنوان المقطع: {section_title}
ملخص المحتوى: {section_content}
"""
        if style_example:
            base_prompt = (
                "Here is an example of the informal style to emulate:\n"
                f"{style_example}\n\n"
                + base_prompt
            )

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert Arabic podcast scriptwriter."},
                {"role": "user", "content": base_prompt},
            ],
        )
        return response.choices[0].message.content.strip()


In [14]:
dialogue_styles = {
    "formal": {
        "host": "Good afternoon, Dr. Patel. Thank you for joining us to discuss the implications of artificial intelligence in public policy.",
        "guest": "Good afternoon, and thank you for the invitation. It’s a pleasure to contribute to this critical conversation."
    },
    "casual": {
        "host": "Hey, Alex! Great to have you on the show. We’ve got a lot to unpack about social media trends today.",
        "guest": "Hey! Super excited to be here. Let’s dive in!"
    },
    "humorous": {
        "host": "Welcome back, folks! Today we’re talking AI—yup, the thing that might steal your job, but can’t steal your coffee. Yet.",
        "guest": "Unless it learns how to queue at Starbucks, I think we’re safe—for now!"
    },
    "educational": {
        "host": "Can you break down what ‘machine learning’ actually means, for listeners who might be new to the concept?",
        "guest": "Of course! Simply put, machine learning is a way for computers to learn patterns from data, kind of like how humans learn from experience."
    },
    "inspirational": {
        "host": "You’ve overcome major obstacles to lead groundbreaking research. What kept you going through the toughest times?",
        "guest": "Honestly, the belief that what we’re building could change lives. That purpose kept me focused."
    }
}

In [15]:
outline_text_clean = outline_text.strip().strip("`")
outline = json.loads(outline_text_clean)

In [16]:
# -------------------- new pipeline using DialogueGenerator --------------------

# 1) pick an optional style exemplar
style_example = dialogue_styles["inspirational"]   # or None for zero-shot

# 2) create the generator
dialogue_generator = DialogueGenerator(deployment)

podcast_dialogues = []

# -------- Intro (use the single "intro" block in the new outline) ------------
intro_text = " ".join([
    outline["intro"]["host_open"],
    outline["intro"]["guest_intro"],
    outline["intro"]["teaser"]
])

intro_dialogue = dialogue_generator.generate_dialogue(
    section_title="المقدمة",
    section_content=intro_text,
    host_persona=host_persona,
    guest_persona=guest_persona,
    tone="ترحيبي",
    style_example=style_example
)
podcast_dialogues.append({"section": "Intro", "dialogue": intro_dialogue})


# ----------------- Core segments (loop over outline["segments"]) -------------
for seg in outline["segments"]:
    seg_content = f"{seg['title']}: {seg['discussion']} " + " ".join(
        [f"Q: {q}" for q in seg["host_questions"]]
    )

    seg_dialogue = dialogue_generator.generate_dialogue(
        section_title=seg["title"],
        section_content=seg_content,
        host_persona=host_persona,
        guest_persona=guest_persona,
        tone=seg.get("emotion_tone", "محايد"),
        style_example=style_example
    )
    podcast_dialogues.append({"section": seg["title"], "dialogue": seg_dialogue})


# --------------------------- Outro -------------------------------------------
outro_text = " ".join([
    outline["outro"]["summary"],
    outline["outro"]["closing_question"],
    outline["outro"]["credits"]
])

outro_dialogue = dialogue_generator.generate_dialogue(
    section_title="الخاتمة",
    section_content=outro_text,
    host_persona=host_persona,
    guest_persona=guest_persona,
    tone="ختامي",
    style_example=style_example
)
podcast_dialogues.append({"section": "Outro", "dialogue": outro_dialogue})

# podcast_dialogues now contains a list of dicts:
# [
#   {"section": "Intro",  "dialogue": "..."},
#   {"section": "<Segment 1>", "dialogue": "..."},
#   ...,
#   {"section": "Outro",  "dialogue": "..."}
# ]


In [17]:
for part in podcast_dialogues:
    print(f"\n--- {part['section']} ---")
    print(part['dialogue'])



--- Intro ---
Host: أهلاً وسهلاً بكم في الحلقة الجديدة من بودكاستنا. أنا مريم أبو زهرة، طالبة صحافة شغوفة بالقصص الإنسانية. اليوم لدينا موضوع ممكن يغير طريقة تفكيرنا في الحياة اليومية ومستقبلنا المشترك. <تنهد> بصراحة، أنا متحمسة جدًا للحلقة اليوم.

Guest: شكراً لكِ يا مريم، وأنا سعيدة جداً بوجودي هنا. الموضوع فعلاً مهم ومثير، والحقيقة إحنا بحاجة نتكلم أكثر عن هذي الأفكار في مجتمعاتنا.

Host: بالضبط، يعني الزراعة الحضرية المستدامة؟ أممم، شو ممكن تعني لنا إحنا اللي نعيش في المدن الكبيرة؟ [pause:1.5s] رانية، بصراحة، ليش اخترتي تخصص حماية الشعاب المرجانية؟ كان غريب بالنسبة لي كيف ترتبط بالأفكار الحضرية!

Guest: <ضحك> سؤال ذكي صراحة! الموضوع فيه رابط أكثر مما نتخيل. الشعاب المرجانية هي مثل المدن تحت الماء، تحتاج للتوازن والمستدامة لتحافظ على حياتها. نفس الشي بالنسبة للزراعة في المدن. لما نعتني بالأرض بطريقة صحيحة، بنخلق بيئة أفضل لكل شيء حوالينا.

Host: واو، يعني الفكرة تعيدنا للبداية. كيف نوفر حياة أفضل لأنفسنا وغيرنا، سواء فوق الأرض أو تحت الماء. <دهشة> بس، هل تحسين الزراعة الحضرية ممكن 

In [18]:
# def modify_dialogues_by_persona(dialogues, persona_criteria):
#     modified_dialogues = []
#     all_dialogues=[]
#     for dialogue in dialogues:
#         all_dialogues.extend(dialogue['dialogue'])
        
#     modified_dialogues = enhancer.enhance_section(
#         "Overall Podcast",
#         all_dialogues,
#         criteria=persona_criteria
#     )
#     return modified_dialogues

# def save_podcast_to_json(podcast, filename="podcast2.json"):
#     with open(filename, 'w') as f:
#         json.dump(podcast, f, indent=4)


In [19]:
# modified_podcast = modify_dialogues_by_persona(podcast_dialogues, "Conversational, friendly, and engaging")

In [20]:
# save_podcast_to_json(modified_podcast)

In [21]:
# modified_podcast